# Dataobjects by example

## Basic

In [4]:
from recordclass import dataobject
import sys

### Empty dataobject

In [5]:
class Empty(dataobject):
     pass

In [6]:
Empty()

Empty()

In [7]:
print('Sizeof:', sys.getsizeof(Empty()))

Sizeof: 16


### Simple 2d data point

In [8]:
class Point(dataobject):
    x:int
    y:int

In [9]:
p1 = Point(1,2)
p2 = Point(x=1, y=2)
print(p1, p2)
p1 == p2

Point(x=1, y=2) Point(x=1, y=2)


True

But
> Only instances of the same class may be equal.

By default instances are mutable:

In [10]:
p = Point(1,2)
p.x = 100
p.y = 200
print(p)

Point(x=100, y=200)


There is an option `readonly=True` for immutability of the instances:

In [11]:
class PointRO(dataobject):
    x:int
    y:int
    __options__ = {'readonly':True}

In [12]:
p = PointRO(1,2)
try:
    p.x = 100
except TypeError as e:
    print(e)

item is readonly


For fast instance creation one may use `fast_new` option:

In [13]:
class FastPoint(dataobject, fast_new=True):
    x:int
    y:int

The following timings explain the advantage of `fast_new` option:

In [14]:
%timeit l1 = [Point(i,i) for i in range(100000)]
%timeit l2 = [FastPoint(i,i) for i in range(100000)]

22.3 ms ± 629 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.93 ms ± 696 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


By default dataobject subclasses are iterable:

If one want to disable iterability of the Point instances, then one have to use `iterable=False` argument:

In [16]:
class PointNI(dataobject):
    x:int
    y:int

In [17]:
try:
    [v for v in PointNI(1,2)]
except TypeError as e:
    print(e)

'PointNI' object is not iterable


### RGB colors

Define class for representation of RGB colors:

In [18]:
class RGB(dataobject):
    r:int = 0
    g:int = 0
    b:int = 0

One may define default values:

In [19]:
p1 = RGB(128, 64)
p2 = RGB(r=128, g=64)
print(p1)
print(p1 == p2)

RGB(r=128, g=64, b=0)
True


Onw have to note that fields without default values have to appear first. Here is invalid definition when the field without default value appears after the field with default value:

In [20]:
try:
    class PointInvalidDefaults(dataobject):
        x:int = 0
        y:int
except TypeError as e:
    print(e)

A field without default value appears after a field with default value


In [22]:
from recordclass.typing import RecordClass

In [23]:
class Point(RecordClass):
    x:int
    y:int
        

In [24]:
sys.getsizeof(Point(1,2))

32

In [28]:
class A(dataobject, fast_new=True):
    x:int
    y:int
    
    def __init__(self, x, y):
        if not (isinstance(x, int) and isinstance(y, int)):
            raise TypeError('x,y are not int')
            
class B:
    __slots__ = 'x', 'y'
    x:int
    y:int
    
    def __init__(self, x, y):
        if not (isinstance(x, int) and isinstance(y, int)):
            raise TypeError('x,y are not int')
        self.x = x
        self.y = y


In [29]:
%timeit l1 = [A(1,2) for i in range(1000000)]

177 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit l2 = [B(1,2) for i in range(1000000)]

228 ms ± 594 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
